## Grabbing historical weather data for 2018

In [2]:
##load variables

import os
from dotenv import load_dotenv
from darksky import forecast
from datetime import date,datetime, timedelta
import pandas as pd


##read in secret keys stored in .env file
load_dotenv('.env')
SECRET = os.environ['dark_sky_secret']
LAT = os.environ['LAT']
LONG = os.environ['LONG']


In [8]:
##test api call
t = datetime(2013, 5, 6, 12).isoformat()
data = forecast(SECRET,LAT,LONG, time=t)
data.time

1367856000

In [68]:
##api call

# sdate = datetime(2018, 1, 1,12)  # start date
# edate = datetime(2018, 12, 31,12)   # end date

# delta = edate - sdate       # as timedelta
# weather = {}
# for i in range(delta.days + 1):
#     day = sdate + timedelta(days=i)
#     key = day.strftime('%D')
#     day = day.isoformat()
#     test = forecast(SECRET,LAT,LONG, time=day)
#     weather[key] = test


In [108]:
all_dates = []
test_df = pd.DataFrame()
for k, v in weather.items():
    toy_df = pd.DataFrame()
    for i in range(0,22):
        all_dates.append(k)
        timei = pd.DataFrame.from_dict(weather[k]['hourly']['data'][i],orient='index').transpose()
        timei['datetime'] =  pd.to_datetime(weather[k]['hourly']['data'][i]['time'],unit='s')
        toy_df= pd.concat([toy_df,timei], sort = False)
    test_df = pd.concat([test_df, toy_df], sort = False)
        
        

In [109]:
test_df

,time,summary,icon,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windGust,windBearing,cloudCover,uvIndex,visibility,datetime,precipType,precipAccumulation,ozone
0,1514782800,Clear,clear-night,0,0,12.78,12.78,3.53,0.66,1031.28,0.87,2.75,318,0,0,6.172,2018-01-01 05:00:00,NaN,NaN,NaN
0,1514786400,Clear,clear-night,0,0,12.38,12.38,3.33,0.66,1031.02,0.43,1.98,323,0,0,6.377,2018-01-01 06:00:00,NaN,NaN,NaN
0,1514790000,Clear,clear-night,0,0,11.83,11.83,3.34,0.68,1031.23,0.41,1.44,303,0,0,5.569,2018-01-01 07:00:00,NaN,NaN,NaN
0,1514793600,Clear,clear-night,0,0,11.41,11.41,3.44,0.7,1031.51,0.47,1.42,305,0,0,5.632,2018-01-01 08:00:00,NaN,NaN,NaN
0,1514797200,Clear,clear-night,0,0,11.23,11.23,3.62,0.71,1031.64,0.64,1.47,314,0,0,6.812,2018-01-01 09:00:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1546293600,Possible Drizzle,rain,0.0016,0.47,45.17,42.87,45.17,1,1017.73,4.54,6.48,121,1,0,2.067,2018-12-31 22:00:00,rain,NaN,244.2
0,1546297200,Possible Drizzle,rain,0.0027,0.54,45.3,45.3,44.88,0.98,1017.11,1.49,1.67,106,1,0,0.963,2018-12-31 23:00:00,rain,NaN,241.3
0,1546300800,Possible Drizzle,rain,0.0024,0.44,45.53,45.53,45.08,0.98,1016.59,1.76,3.18,143,1,0,0.851,2019-01-01 00:00:00,rain,NaN,240.2
0,1546304400,Foggy,fog,0.0013,0.3,45.59,45.59,44.88,0.97,1015.29,1.31,2.44,170,1,0,1.261,2019-01-01 01:00:00,rain,NaN,236.2


###

In [111]:
# import pickle
# with open('data/weather2018.pkl', 'wb') as fid:
#     pickle.dump(test_df, fid,2) 

In [4]:
import pickle
weather = pickle.load(open('data/weather2018.pkl', 'rb'))
weather = pd.DataFrame(weather)

head= pd.read_csv('data/head.csv')

In [13]:
##test api call
t = pd.to_datetime(head.AppointmentDate[1]).isoformat()
data = forecast(SECRET,LAT,LONG, time=t)
pd.to_datetime(data.time, unit='s')

Timestamp('2018-06-21 13:00:00')

In [6]:


for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    key = day.strftime('%D')
    day = day.isoformat()
    test = forecast(SECRET,LAT,LONG, time=day)
    weather[key] = test


,Unnamed: 0,PatientId,AppointmentDate,newbie,insuranceDummy,guarantorIsPatient,loyalty,noshow,PatientEmail,GuarantorEmail,...,insurance,provider,source,procedure,weekday,apptType,newpatientfile,Age_npf,Patient,AppointmentId
0,0,5308,2018-09-26 14:30:00,0,0,True,27,0,daviddm75@gmail.com,daviddm75@gmail.com,...,Insurance_others,"Provider_Mohammed Almzayyen, DDS",Source_no_reported_source,Procedures_others,Wednesday,Hygiene,201606,NaN,David Morin,0
1,1,9552,2018-06-21 09:00:00,0,1,True,13,0,north.virginia@gmail.com,north.virginia@gmail.com,...,Insurance_ANTHEM BCBS,"Provider_Mohammed Almzayyen, DDS",Source_Google,"Procedures_D0220, D0230, D0140",Thursday,Restorative,201705,30.0,Virginia North,1
2,2,6882,2018-06-12 08:00:00,0,0,True,24,0,missericab88@aol.com,missericab88@aol.com,...,Insurance_others,Provider_Stephanie Douglas,Source_no_reported_source,Procedures_others,Tuesday,Hygiene,201606,NaN,Erica Gibson,2
3,3,9637,2018-02-06 08:00:00,0,0,True,20,0,NaN,NaN,...,Insurance_others,"Provider_Mohammed Almzayyen, DDS",Source_no_reported_source,Procedures_others,Tuesday,Restorative,201606,NaN,Sheerali Abdulrahimzai,3
4,4,10143,2018-06-20 12:00:00,0,0,False,24,0,samadi@vt.edu,SS3AQ@VIRGINIA.EDU,...,Insurance_others,Provider_Rachel Nguyen,Source_no_reported_source,Procedures_D0180,Wednesday,Restorative,201606,NaN,Mohammad Abdolsamadi,4
